# ETL para dataset especifico para la funcion UserForGenre

In [4]:
import pandas as pd
import pyarrow.parquet as pq

### Cargamos los Datasets

In [5]:
steam = pd.read_json('../Datasets/Steam_Games_Limpio.json.gz', compression='gzip')
review = pd.read_json('../Datasets/User_Reviews_Limpio.json.gz', compression='gzip')
items = pq.read_table('Datasets OPS/items.parquet').to_pandas()

#### Nos quedamos con la columna necesaria

In [6]:
generosAVer = steam['genres']

#### Guardamos en listaDeGeneros todos los generos que existen en el dataset

In [7]:
listaDeGeneros = []
for linea in generosAVer:
  for i in linea:
    if i not in listaDeGeneros:
      listaDeGeneros.append(i)

generos = {'genero':listaDeGeneros}
df = pd.DataFrame(generos)
df[['usuario','Horas jugadas']] = None

#### Vemos los items de todos los usuarios

In [8]:
user_items = items # o pd.read_json('API/Datasets/items.json') 
itemsDeUsuario = items.explode('items')
userID = itemsDeUsuario['user_id']
itemsDeUsuario = pd.json_normalize(itemsDeUsuario['items'])
itemsDeUsuario['user_id'] = list(userID)
itemsDeUsuario.drop(columns=['playtime_2weeks','item_name'], inplace=True)
itemsDeUsuario.dropna(subset='item_id', inplace=True)

#### Unimos los items de todos los usuarios con los que tenemos informacion en el dataset de steamgames

In [9]:
itemsDeUsuario['item_id'] = itemsDeUsuario['item_id'].astype(int)
itemsDeUsuario = itemsDeUsuario.merge(steam[['id','genres','release_date']], left_on='item_id', right_on='id')
itemsDeUsuario

,item_id,playtime_forever,user_id,id,genres,release_date
0,10,6.0,76561197970982479,10,[Action],2000
1,10,0.0,js41637,10,[Action],2000
2,10,0.0,Riot-Punch,10,[Action],2000
3,10,93.0,doctr,10,[Action],2000
4,10,108.0,corrupted_soul,10,[Action],2000
...,...,...,...,...,...,...
4186243,456780,6.0,peter1023199,456780,"[Racing, Simulation, Sports]",2016
4186244,444770,46.0,76561198071808318,444770,"[Adventure, Indie]",2016
4186245,375450,51.0,76561198081168593,375450,[Strategy],2016
4186246,354280,164.0,76561198107283457,354280,"[Action, Casual, Indie, Simulation]",2016


#### Calculamos por genero cual es el usuario que mas jugo a tal genero, y la cantidad de horas por año de lanzamiento del juego que tuvo

In [10]:
for i in df['genero'].values:
    genero = i

    #Juegos del genero pedido
    mascara = itemsDeUsuario['genres'].apply(lambda x: genero in x)

    #Calcular las horas que tiene un usuario con juegos del genero
    sumaDehoras = itemsDeUsuario[mascara].groupby('user_id')['playtime_forever'].sum().sort_values(ascending=False)
    
    if not sumaDehoras.empty:
        usuario = sumaDehoras.head(1).index[0]
    else:
        usuario = None

    df.loc[df['genero'] == genero, 'usuario'] = usuario

    aniosYhoras = itemsDeUsuario[(itemsDeUsuario['user_id'] == usuario) & mascara].groupby('release_date')['playtime_forever'].sum()
    anio = aniosYhoras.index
    horasJugadas = {}
    for horas, anio in enumerate(anio):
        horasJugadas[f'Año {anio}'] = f'Horas: {aniosYhoras.iloc[horas]}'

    df.loc[df['genero'] == genero, 'Horas jugadas'] = [horasJugadas]


#### Hacemos unos ultimos retoques en el data set para exportarlo y poder utilizarlo en la funcion

In [11]:
df.dropna(subset=usuario, inplace=True)
df.reset_index(drop=True,inplace=True)

In [12]:
df['genero'] = df['genero'].apply(lambda x: x.capitalize())

#### Exportamos el dataset para su posterior uso en la API

In [13]:
df.to_json(".../Datasets/genre.json.gz", compression='gzip')